<a href="https://colab.research.google.com/github/LucasBrandaoGomes/Projeto-ETL-Consultas-SQL/blob/main/G5_CO2_Completo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##PANDAS



Instalação do pacote gcsfs para acesso de arquivos direto do GCStorage 

In [ ]:
pip install gcsfs

Instalação do pacote Pandera para validação de dados

In [ ]:
pip install pandera

Importação de biblioteca para autenticação da service account e uso dos arquivos do bucket GCP

In [ ]:
import os

Foi criada uma conta de serviço relacionada ao Bucket do CloudStorage bem como uma chave para interação entre as ferramentas. A seguir o arquivo de chave da conta de servico é atribuida a variável serviceAccount.

In [ ]:
serviceAccount ='/content/chave-projeto-g5.json'

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

Importação de bibliotecas a serem utilizadas na etapa de tratamento pandas/pandera.

In [ ]:
import pandas as pd
import pandera as pa
from google.cloud import storage


Leitura do arquivo CSV utilizando o URI do arquivo no bucket.

In [ ]:
df = pd.read_csv('gs://projeto-final-bucket-g5/entrada/Emissao_CO2_pais.csv')
df

,Entity,Code,Year,Annual CO₂ emissions (tonnes )
0,Afghanistan,AFG,1949,14656.00
1,Afghanistan,AFG,1950,84272.00
2,Afghanistan,AFG,1951,91600.00
3,Afghanistan,AFG,1952,91600.00
4,Afghanistan,AFG,1953,106256.00
...,...,...,...,...
20848,Zimbabwe,ZWE,2013,11536239.29
20849,Zimbabwe,ZWE,2014,11866348.41
20850,Zimbabwe,ZWE,2015,10907603.94
20851,Zimbabwe,ZWE,2016,9932649.88


In [ ]:
#verificação do tipo do dado
df.dtypes

Entity                             object
Code                               object
Year                                int64
Annual CO₂ emissions (tonnes )    float64
dtype: object

In [ ]:
#função pd.unique sendo utilizada para verificarmos as lista de países(Entity)
pd.unique(df['Entity'])

Utilizando a função pd.unique verificamos que alguns paises não são de interesse para o propósito da análise ou apresentam divergências.

* As linhas referentes seguintes países/territórios serão excluidas:

[Africa, Americas (other), Antarctic Fisheries', 'Asia and Pacific (other)', 'Bonaire Sint Eustatius and Saba','Bermuda','Brunei', 'EU-28','Cook Islands', 'Europe (other)', 'Gibraltar', 'International transport', 'Kyrgysztan', 'Middle East', 'Maldives', 'Marshall Islands', 'Nauru', 'Saint Helena', 'Saint Pierre and Miquelon', 'Sint Maarten (Dutch part)', 'Statistical differences', 'South Sudan', 'Tuvalu',  'Vanuatu', 'Wallis and Futuna Islands', 'World','Czechoslovakia'.

* Os paises com os seguintes nomes terão nomes ajustados para alinhar com o outro dataset: 

['Timor'= 'Timor leste'
'Antarctic Fisheries'  = 'Antarctic'
'Faeroe Islands = 'Faroe Islands'
'Falkland Islands', = 'Islas Malvinas'
'Micronesia (country)' = 'Micronesia' 
'Democratic Republic of Republic of the Congo'	 = 'Democratic Republic of the Congo'
'Republic of the Congo' = 'Congo']

In [ ]:
len(df['Entity'].unique())

233

Realização de Backup

In [ ]:
#definição de um fataframe de backup
df_backup = df.copy()

In [ ]:
df_backup

Tradução de colunas

In [ ]:
(df.rename(columns={'Entity': 'País', 'Code': 'Código', 
                       'Year': 'Ano', 'Annual CO₂ emissions (tonnes )': 'Emissão CO₂ (ton/ano)'}
                        , inplace=True))

Eliminando linhas relacionadas a datas superiores a 2012. Tal tratamento foi feito analisando o outro dataset (Temperatura média), a fim de igualar os períodos de tempo entre os conjunto de dados.

In [ ]:
filtro_anomax = df.Ano <= 2012

df = df.loc[filtro_anomax]

Verificação dados nulos

In [ ]:
df.isna().sum()

País                        0
Código                   2162
Ano                         0
Emissão CO₂ (ton/ano)       0
dtype: int64

In [ ]:
#conferindo inconsistências
pd.unique(df['Código'])

In [ ]:
#vizualizando os valores NaN
filter = df['Código'].isna()
df.loc[filter]

,País,Código,Ano,Emissão CO₂ (ton/ano)
69,Africa,NaN,1751,0.0
70,Africa,NaN,1752,0.0
71,Africa,NaN,1753,0.0
72,Africa,NaN,1754,0.0
73,Africa,NaN,1755,0.0
...,...,...,...,...
20343,Wallis and Futuna Islands,NaN,2008,21984.0
20344,Wallis and Futuna Islands,NaN,2009,29312.0
20345,Wallis and Futuna Islands,NaN,2010,29312.0
20346,Wallis and Futuna Islands,NaN,2011,25648.0


Após conferir lista de países entre os bancos de dados a serem analisados, foram feitas as exclusões de linhas referentes aos países não relevantes.

In [ ]:
df.drop(df[(df['País'] == 'Africa')].index, inplace=True)
df.drop(df[(df['País'] == 'Americas (other)')].index, inplace=True)
df.drop(df[(df['País'] == 'Asia and Pacific (other)')].index, inplace=True)
df.drop(df[(df['País'] == 'Bonaire Sint Eustatius and Saba')].index, inplace=True)
df.drop(df[(df['País'] == 'Bermuda')].index, inplace=True)
df.drop(df[(df['País'] == 'Brunei')].index, inplace=True)
df.drop(df[(df['País'] == 'EU-28')].index, inplace=True)
df.drop(df[(df['País'] == 'Europe (other)')].index, inplace=True)
df.drop(df[(df['País'] == 'Cook Islands')].index, inplace=True)
df.drop(df[(df['País'] == 'Gibraltar')].index, inplace=True)
df.drop(df[(df['País'] == 'International transport')].index, inplace=True)
df.drop(df[(df['País'] == 'Kyrgysztan')].index, inplace=True)
df.drop(df[(df['País'] == 'Middle East')].index, inplace=True)
df.drop(df[(df['País'] == 'Maldives')].index, inplace=True)
df.drop(df[(df['País'] == 'Marshall Islands')].index, inplace=True)
df.drop(df[(df['País'] == 'Nauru')].index, inplace=True)
df.drop(df[(df['País'] == 'Saint Helena')].index, inplace=True)
df.drop(df[(df['País'] == 'Saint Pierre and Miquelon')].index, inplace=True)
df.drop(df[(df['País'] == 'Sint Maarten (Dutch part)')].index, inplace=True)
df.drop(df[(df['País'] == 'Statistical differences')].index, inplace=True)
df.drop(df[(df['País'] == 'South Sudan')].index, inplace=True)
df.drop(df[(df['País'] == 'Tuvalu')].index, inplace=True)
df.drop(df[(df['País'] == 'Vanuatu')].index, inplace=True)
df.drop(df[(df['País'] == 'Wallis and Futuna Islands')].index, inplace=True)
df.drop(df[(df['País'] == 'Czechoslovakia')].index, inplace=True)
df.drop(df[(df['País'] == 'World')].index, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Existem 21 valores nulos na coluna de código, e são referentes ao território da Antartica. Como o dataset de variação de temperatura não apresenta medições para este território, preferiu-se eliminar também aqui.

In [ ]:
df.isna().sum()

País                      0
Código                   21
Ano                       0
Emissão CO₂ (ton/ano)     0
dtype: int64

In [ ]:
df.drop(df[(df['País'] == 'Antarctic Fisheries')].index, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
#filtro utilizado para verificar se as mudanças foram efetuadas
filtro=df.País=='Antarctic Fisheries'
df.loc[filtro]

,País,Código,Ano,Emissão CO₂ (ton/ano)


Alteração de nomes de países para futuramente realizar operações utilizando joins.

In [ ]:
df.loc[df['País'] == 'Timor' , ['País']] = 'Timor Leste'
df.loc[df['País'] == 'Republic of the Congo' , ['País']] = 'Congo'
df.loc[df['País'] == 'Faeroe Islands' , ['País']] = 'Faroe Islands'
df.loc[df['País'] == 'Falkland Islands' , ['País']] = 'Islas Malvinas'
df.loc[df['País'] == 'Micronesia (country)' , ['País']] = 'Micronesia'
df.loc[df['País'] == 'Congo (Democratic Republic of the)' , ['País']] = 'Democratic Republic of the Congo'


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


In [ ]:
pd.unique(df['País'])

Importação do arquivo em formato CSV para o bucket

In [ ]:
client=storage.Client()
#a variável bucket recebe o nome do bucket do cloud storage
bucket=client.get_bucket('projeto-final-bucket-g5')
#caminho de destino, pasta saída, o qual será salvo o arquivo em formato csv
bucket.blob('saida/df_emissoes_pandas_tratamento1.csv').upload_from_string(df.to_csv(index=False), 'text/cvs')

O arquivo a cima foi salvo no intuito de utilizar a ferramenta DataPrep para ter uma melhor vizualização dos dados e assim continuar com os tratamentos necessarios.

* Foram analisados os valores de emissão de CO2 e de Temperatura (no dataset de temperatura) e definido um intervalo de anos a serem analisados (1900 - 2012).

In [ ]:
#Definição do intervalo após ano de 1900
filtro_anomin = df.Ano >= 1900
df=df.loc[filtro_anomin]

Validação com Pandera

In [ ]:
#construção da base de comparação para validação
schema = pa.DataFrameSchema(
    columns = {  
        "País":pa.Column(pa.String),
        "Código":pa.Column(pa.String,pa.Check.str_length(3,3), nullable=True),
        "Ano":pa.Column(pa.Int),
        "Emissão CO₂ (ton/ano)":pa.Column(pa.Float, nullable=True)
        }
)

In [ ]:
schema.validate(df)

,País,Código,Ano,Emissão CO₂ (ton/ano)
0,Afghanistan,AFG,1949,14656.00
1,Afghanistan,AFG,1950,84272.00
2,Afghanistan,AFG,1951,91600.00
3,Afghanistan,AFG,1952,91600.00
4,Afghanistan,AFG,1953,106256.00
...,...,...,...,...
20843,Zimbabwe,ZWE,2008,7718874.78
20844,Zimbabwe,ZWE,2009,5519508.33
20845,Zimbabwe,ZWE,2010,7681010.98
20846,Zimbabwe,ZWE,2011,9439850.98


Arquivo salvo diretamento para o Bucket (Data Lake) da Google Cloud Storage.

In [ ]:
client=storage.Client()
bucket=client.get_bucket('projeto-final-bucket-g5')
bucket.blob('saida/df_emissoes_pandas_tratado_final.csv').upload_from_string(df.to_csv(index=False), 'text/cvs')

##PYSPARK

Instalação da pacote Pyspark

In [ ]:
!pip install pyspark

Importação de módulos e de funções específicas dentro da biblioteca Pyspark

In [ ]:
from pyspark.sql import SparkSession
#ParkSession é uma string de conexão, na qual serão definidos os devidos parâmetros
import pyspark.sql.functions as F
#importação da biblioteca de funções do pacote spark atribuindo F para o uso durante o código
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, FloatType, DoubleType, DateType, ArrayType, DoubleType, BooleanType
#importação dos tipos de variaveis para futura estruturação e validação
from pyspark.sql.functions import col,array_contains
from pyspark.sql.window import Window
#importação de funções do tipo window

Configurando a SparkSession

In [ ]:
spark = SparkSession.builder\
        .master("local")\ #master: local, cluester, ou local onde irá rodar a sessão
        .appName("dataframe_emissoes")\#identificaçao do aplicação a ser criad
        .config("spark.ui.port","4050")\#configurção via API com apache spark, e definção da porta UI, que no caso do colab é 4050
        .getOrCreate()#iniciar a sessão

Criando o Esquema/Schema com a função StructType, usando o dataframe já criado na etapa do Pandas. Aqui, apesar da validação feita anteriormente, é interessante o uso do StructType para grantir os tipos das variáveis no momento da criação do dataframe no pyspark, assim a efetuação de calculos e manipulações corretamente.

In [ ]:
schema = StructType() \
      .add("País",StringType(),False) \
      .add("Código", StringType(), True) \
      .add("Ano", IntegerType(), False) \
      .add("Emissão CO₂ (ton/ano)", FloatType(), True)     
   

dfCO2_spark = spark.createDataFrame(df, schema=schema)
#aqui o df da etapa anterior foi passado como parâmetro de dados
dfCO2_spark.printSchema()

root
 |-- País: string (nullable = false)
 |-- Código: string (nullable = true)
 |-- Ano: integer (nullable = false)
 |-- Emissão CO₂ (ton/ano): float (nullable = true)



In [ ]:
dfCO2_spark.show()

+-----------+------+----+---------------------+
|       País|Código| Ano|Emissão CO₂ (ton/ano)|
+-----------+------+----+---------------------+
|Afghanistan|   AFG|1949|              14656.0|
|Afghanistan|   AFG|1950|              84272.0|
|Afghanistan|   AFG|1951|              91600.0|
|Afghanistan|   AFG|1952|              91600.0|
|Afghanistan|   AFG|1953|             106256.0|
|Afghanistan|   AFG|1954|             106256.0|
|Afghanistan|   AFG|1955|             153888.0|
|Afghanistan|   AFG|1956|             183200.0|
|Afghanistan|   AFG|1957|             293120.0|
|Afghanistan|   AFG|1958|             329760.0|
|Afghanistan|   AFG|1959|             384571.4|
|Afghanistan|   AFG|1960|             413883.4|
|Afghanistan|   AFG|1961|             490797.7|
|Afghanistan|   AFG|1962|            688594.25|
|Afghanistan|   AFG|1963|             706736.0|
|Afghanistan|   AFG|1964|             838550.8|
|Afghanistan|   AFG|1965|            1006916.5|
|Afghanistan|   AFG|1966|            109

A célula abaixo é responsável pela criação da coluna Diferença Emissões (ton), a qual é preenchida com a função de janela 'lag', que preenche cada elemento fazendo a diferença da emissão de CO2 do ano seguinte em relação ao ano anterior. 

In [ ]:
lag_window = Window.partitionBy('País').orderBy('País','Ano')
#criação do que chamamos de window, que é uma partição dos dados a qual será submetida ao uso da função 'lag' para criação da coluna de Diferença Emissções (ton)

dfCO2_spark = (
        dfCO2_spark.select('País', 'Código','Ano', 'Emissão CO₂ (ton/ano)')
        .withColumn('Diferenca Emissões(ton)', F.col('Emissão CO₂ (ton/ano)') - F.lag('Emissão CO₂ (ton/ano)')
        .over(lag_window))
)
dfCO2_spark.show()

+-----------+------+----+---------------------+-----------------------+
|       País|Código| Ano|Emissão CO₂ (ton/ano)|Diferenca Emissões(ton)|
+-----------+------+----+---------------------+-----------------------+
|Afghanistan|   AFG|1949|              14656.0|                   null|
|Afghanistan|   AFG|1950|              84272.0|                69616.0|
|Afghanistan|   AFG|1951|              91600.0|                 7328.0|
|Afghanistan|   AFG|1952|              91600.0|                    0.0|
|Afghanistan|   AFG|1953|             106256.0|                14656.0|
|Afghanistan|   AFG|1954|             106256.0|                    0.0|
|Afghanistan|   AFG|1955|             153888.0|                47632.0|
|Afghanistan|   AFG|1956|             183200.0|                29312.0|
|Afghanistan|   AFG|1957|             293120.0|               109920.0|
|Afghanistan|   AFG|1958|             329760.0|                36640.0|
|Afghanistan|   AFG|1959|             384571.4|              548

Salvando o arquivo no Google Drive

In [ ]:
dfCO2_spark.write.format("csv") \
.option("header", "true") \
.option("inferschema", "true") \
.option("delimiter", ",") \
.save("/content/drive/MyDrive/dados_soulcode/dfCO2.csv")

Montagem da pasta /drive para poder acessar a pasta /MyDrive, onde o arquivo 
com o dataframe tratado com pyspark estava salvo.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Definição de função, para poder salvar o arquivo pyspark diretamente para o Bucket/Data Lake, utilizando como intermédio o ambiente do drive

In [ ]:
def upload_blob(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # The ID of your GCS bucket
    # bucket_name = "your-bucket-name"
    # The path to your file to upload
    # source_file_name = "local/path/to/file"
    # The ID of your GCS object
    # destination_blob_name = "storage-object-name"

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)

    blob.upload_from_filename(source_file_name)

    print(
        "File {} uploaded to {}.".format(
            source_file_name, destination_blob_name
        )
    )

Arquivo salvo no Bucket/Data Lake:

In [ ]:
upload_blob("projeto-final-bucket-g5", "/content/drive/MyDrive/dados_soulcode/dfCO2.csv/dfCO2pyspark.csv", "saida/df_sparkCO2_emissoes_pyspark_final.csv")


File /content/drive/MyDrive/dados_soulcode/dfCO2.csv/dfCO2pyspark.csv uploaded to saida/df_sparkCO2_emissoes_pyspark_final.csv.
